In [ ]:
import pandas as pd
import numpy as np
import os
from ccij_clean import create
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = None

In [ ]:
cwd = os.getcwd().strip('python_files')
cwd

In [ ]:
comb = create.create()
df = pd.read_csv(cwd+'data/raw_csv_files/casenotes/case_comments.csv', usecols=['record_id', 'caseaction_id', 'actionDate', 'created_by_id', 'comment', 'caseevent_id'])
transfer, gsa, golden = pd.read_csv(cwd+'data/raw_csv_files/casenotes/transfer.csv', usecols=['record_id', 'caseaction_id', 'actionDate', 'created_by_id', 'comment', 'caseevent_id']), pd.read_csv(cwd+'data/raw_csv_files/casenotes/gsa.csv', usecols=['record_id', 'caseaction_id', 'actionDate', 'created_by_id', 'comment', 'caseevent_id']), pd.read_csv(cwd+'data/raw_csv_files/casenotes/golden.csv', usecols=['record_id', 'caseaction_id', 'actionDate', 'created_by_id', 'comment', 'caseevent_id'])
df2 = pd.concat([transfer, gsa, golden])
df2 = df2.loc[~df2.caseevent_id.duplicated(), :]
df2.caseaction_id.replace(to_replace='@MVDF', value='Transfer', inplace=True)


In [ ]:
df = pd.concat([df,df2])

In [ ]:
comb.head()

In [ ]:
ids = pd.DataFrame(comb.loc[:,'record_id'])
ids = ids.loc[~ids.record_id.duplicated(),:]

In [ ]:
df['record_id'] = df['record_id'].astype('str')

In [ ]:
merged = pd.merge(ids, df, on='record_id')

In [ ]:
merged.rename(columns = {'actionDate':'action_date', 'created_by_id':'created_by'}, inplace=True)


In [ ]:
merged.tail()

In [ ]:
ids.shape

In [ ]:
merged.caseaction_id.value_counts()

In [ ]:
merged.created_by = merged.created_by.fillna(value='AUTO_ADMIN')

**going to drop a lot of caseaction types, keeping the essentials for now**

In [ ]:
keep = ['Case Note', 'Legal Help (other)', 'Legal Call/Consult', 'Legal Consult w/Atty','Referred'
                              'Attorney Retained', 'Placed w/ Pro Bono','Franco (Maybe)', 'Transfer', 'Task', '@Yuba','Detention Conditions','Repeat CCIJ Hotline','AOD Repeat', 'CALL-UP']
drop = ['Hearing Date', 'Master Hearing', "@MVDF",
        'Individual Hearing','Filing','Other Hearing Type','Court Hearing','Document Received','Repeat Intake','Client Meeting',
        'Client Meeting','Deadline', 'Intake Form','@Golden State','Repeat CCIJ Hotline','AOD Repeat','@Yuba','CALL-UP','Task','PRAP','AILA Case Summary','@Otay Mesa',
                        'CrimmHistory','ICE Appointment','Screened','@Imperial','@Adelanto','@Desert View Annex']

In [ ]:
#df1 = merged.loc[~merged['caseaction_id'].isin(drop),:]
df1 = merged

**relabeling legal call consult to legal consult w/ atty**

In [ ]:
df1.replace(to_replace='Legal Call/Consult', value='Legal Consult w/Atty', inplace=True)

**relabeling for better exporting later**

In [ ]:
df1.replace(to_replace='Legal Consult w/Atty', value='Legal Consult with an Attorney', inplace=True)
df1.replace(to_replace='Placed w/ Pro Bono', value='Placed with Pro Bono', inplace=True)
df1.replace(to_replace='Franco (Maybe)', value='Franco - Maybe', inplace=True)
df1.replace(to_replace='Health Advocacy', value='Detention Conditions', inplace=True)
#df1.replace(to_replace='Legal Help (other)', value='Case Note', inplace=True)
df1.replace(to_replace='VEED Task', value='VEED', inplace=True)
df1.replace(to_replace='VEED (vaccine) flag', value='VEED', inplace=True)
df1.replace(to_replace=drop,value='Case Note', inplace=True)
df1.replace(to_replace=r'(\*\*)', value='',regex=True,inplace=True)
df1.replace(to_replace=r'(\r\n\r\n)', value=' ',regex=True,inplace=True)
df1.replace(to_replace=r'(\r\n)', value=' ',regex=True,inplace=True)

In [ ]:
df1.caseaction_id.value_counts()

In [ ]:
df1 = df1.sort_values(by='record_id')
df1 = df1.reset_index()
df1['unique_id'] = df1['record_id'].astype(str)
df1.drop(labels=['index','caseevent_id'], axis=1, inplace = True)
df1 = df1[['record_id','unique_id','caseaction_id', 'action_date', 'created_by', 'comment']]

In [ ]:
df1.tail()

In [ ]:
lh = df1[df1['caseaction_id']=='Legal Help (other)']

In [ ]:
m1 = lh[lh['comment'].str.startswith(('mail','Mail','mailed','Mailed','send','Send','sent','Sent'))]
m2 = lh[lh['comment'].str.contains(r'(?i)Legal Assistance Provided.*mail|Legal Assistance Provided.*send|Legal Assistance Provided.*sent|Legal Assistance Provided.*provided', regex=True)]
m3 = lh[lh['comment'].str.contains(r'(?i)Legal Assistance Provided.*kk emailed|Legal Assistance Provided.*kk spoke|Legal Assistance Provided.*kk texted|Legal Assistance Provided.*kk forwarded|Legal Assistance Provided.*kk submitted|Legal Assistance Provided.*kk called|Legal Assistance Provided.*Dad natz|Legal Assistance Provided.*kk requested|Legal Assistance Provided.*kk tried|Legal Assistance Provided.*kk observed|Legal Assistance Provided.*kk completed|Legal Assistance Provided.*kk left', regex=True)]
m4 = lh[lh['comment'].str.startswith(('Mailed: Drafted letter for Ricardo','Mailed: NOA and fee waiver to BIA','Mailed: KK emailed in brief on appeal','Mailed: KK mailed BIA signed copy of NOA','Mailed: KK email filed translated declaration w/ EOIR','Other: We translated 2 denuncias','Contacted Family: KK had long','Mailed: Email filed bond evidence','Mailed: LK spoke w/Carlos.','Mailed: Friend of court letter','Legal Assistance Provided Emailed ERO, see below from','Researched, Called and Emailed','Legal Assistance Provided - KK rcvd. fax','Contacted Family: KK called & emailed daughter','Legal Assistance Provided Emailed DO Kroman','Legal Assistance Provided Monica spoke','Legal Assistance Provided - his dad texted KK on Friday','Legal Assistance Provided - KK sent bullet point','Legal Assistance Provided SM spoke to','Legal Assistance Provided: TK spoke with Omar','Legal Assistance Provided SM talked to Omar and told','Legal Assistance Provided Steph spoke to', 'Legal Assistance Provided - KK Priority Mailed Re-Submission', 'Legal Assistance Provided - KK Priority Mailed Re-Submission','Mailed: LK spoke w/Carlos.','Legal Assistance Provided Emailed ERO, see below from:','Legal Assistance Provided Steph'))]

In [ ]:
m = lh.loc[(m1.index|m2.index),:]
m = m.loc[~m.index.isin(m3.index),:]
m = m.loc[~m.index.isin(m4.index),:]

In [ ]:
df1.loc[m.index,'caseaction_id']='Mailed Items'

In [ ]:
df1.caseaction_id.replace(to_replace='Legal Help (other)',value='Prose+ and Legal Services', inplace=True)

In [ ]:
df1.caseaction_id.value_counts()

In [ ]:
casetypes = list(df1.caseaction_id.value_counts().index)

In [ ]:
d = {casetype: df1[df1['caseaction_id']==casetype] for casetype in casetypes}

In [ ]:
for name, frame in d.items():
    print(name)

In [ ]:
def casenotes(df):
    casetypes = list(df.caseaction_id.value_counts().index)
    d = {casetype: df[df['caseaction_id']==casetype] for casetype in casetypes}
    for name, frame in d.items():
        frame = frame.set_index('unique_id')
        frame.index = frame.index + ' - ' + (frame.groupby(level=0).cumcount().astype(int)+1).astype(str)
        frame = frame.reset_index()
        frame.columns = frame.columns.str.replace('index', 'unique_id')
        d[name] = frame
    return d

In [ ]:
casenotes = casenotes(df1)

In [ ]:
casenotes['Attorney Retained'].unique_id.value_counts()

In [ ]:
def create_csv(casenotes):
    for name, df in casenotes.items():
        df.to_csv(cwd+'data/exported_files/casenotes/' + str(name) +'.csv')

In [ ]:
create_csv(casenotes)

In [ ]:
fix = casenotes['Case Note']

In [ ]:
fix[fix['action_date']>'2022-10-03'].sort_values('action_date').to_csv('/Users/stjames/Dropbox/Pablo/casenote_delta.csv')